# BigData Final Project | Steam
## <font color = 'blue'> Notebook 1 | Clean Data _ Part 1 </font>
### Team Member: Jim Fang, WooJong Choi, Han Jeon, Tam Nguyen
June 2020
___

### Steam Data
https://steam.internet.byu.edu/#


### Clean & add column to data

---
## I. Import Libraries

In [1]:
from pyspark import SparkContext, SparkConf
from pyspark.context import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import pyspark.sql.types as t
from pyspark.sql.functions import broadcast
from pyspark.sql.functions import regexp_replace
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import isnan, when, count, col, size
from functools import reduce

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
spark = SparkSession.builder.enableHiveSupport().appName('ReadWriteData').getOrCreate()
sc = spark.sparkContext

In [3]:
!hdfs dfs -ls /user/tamng/jwht/SteamData

Java HotSpot(TM) 64-Bit Server VM warning: ignoring option MaxPermSize=512M; support was removed in 8.0
Found 7 items
-rwxrwxrwx   3 tamng   tamng 5116243481 2020-05-09 22:16 /user/tamng/jwht/SteamData/Games2_100milrows.csv
-rwxrwxrwx   3 tamng   tamng   79545914 2020-05-09 22:16 /user/tamng/jwht/SteamData/app_id_info_join.csv
-rwxrwxrwx   3 tamng   tamng 1045968424 2020-05-09 22:17 /user/tamng/jwht/SteamData/friends_subset.csv
drwxr-xr-x   - hanjeon tamng          0 2020-05-17 22:42 /user/tamng/jwht/SteamData/game2_df.csv
-rwxrwxrwx   3 tamng   tamng   83366366 2020-05-09 22:16 /user/tamng/jwht/SteamData/groups_subset.csv
-rwxrwxrwx   3 tamng   tamng     568119 2020-05-09 22:16 /user/tamng/jwht/SteamData/metacritic_games.csv
-rwxrwxrwx   3 tamng   tamng  899281516 2020-05-09 22:16 /user/tamng/jwht/SteamData/player_summaries_subset.csv


In [4]:
!hdfs dfs -ls /user/tamng/jwht/code

Java HotSpot(TM) 64-Bit Server VM warning: ignoring option MaxPermSize=512M; support was removed in 8.0
Found 1 items
-rwxrwxrwx   3 tamng tamng     106856 2020-05-17 12:21 /user/tamng/jwht/code/1.JoinData.ipynb


In [ ]:
def check_missing(df):
    ''' Check missing value'''
    df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns]).show()

In [ ]:
def rename_col(df, newColumns):
    ''' Rename all columns
        df - dataset
        newColumns - list of columns name '''
    oldColumns = df.schema.names
    df = reduce(lambda df, idx: df.withColumnRenamed(oldColumns[idx], newColumns[idx]), range(len(oldColumns)), df)
    return df

In [ ]:
def basic_info(df):
    
    '''
        Print out the basic ddescription for each table, icluding:
        1. total rows/ observation
        2. Check missing value by columns
        3. Print out the first 3 lines
        4. Basic description
    '''
    
    print('TOTAL ROWS:', df.count())
    print('\n')
    print('*-------------'*5)
    print('\n')
    print('MISSING VALUE:')
    check_missing(df)
    print('*-------------'*5)
    print('\n')
    print('PRINT OUT THE 1st 3 LINES:')
    df.show(3, truncate = True)
    print('*-------------'*5)
    print('\n')
    print('TABLE BASIC DESCRIPTION:')
    df.describe().show(10,truncate = True)
    print('*-------------'*5)
    distinct_count = []
    column_name = df.columns
    for i in column_name:
        distinct_count.append(df.select(col(i)).distinct().count())

    print('DISTINCT COUNT BY COLUMN:')
    print('\n')
    print(pd.DataFrame(zip(column_name,distinct_count)).\
      rename(columns={0:'column_name', 1:'distinct_count'}))

---
## II. Import Dataset

### 1. 1st dataset: game2.csv

Expected final table structure: 

|No.|Column | Type | Description
|--|--|--|--|
|1|steam_id|int| The steam ID of the user in question|
|2|app_id|int|The ID of a given app in the user's library|
|3|playtime_2weeks|int|The total time the user has run this app in the two-week period leading up to when this data was requested from the API. Values are given in minutes.|
|4|playtime_forever|int|The total time the user has run this app since adding it to their library. Values are given in minutes.|
|5|dateretrieved|timestap|Timestamp of the time when this game data was requested from the API|

__NOTES__

- Need to create the schema with 5 columns, otherwise, it only input 3 cols and we will miss 1.8 mil rows

In [4]:
game2_schema = t.StructType([t.StructField('_c0', t.StringType(), False), 
                       t.StructField('_c1', t.IntegerType(), False),
                       t.StructField('_c2', t.StringType(), False),
                       t.StructField('_c3', t.StringType(), False),
                       t.StructField('_c4', t.StringType(), False)])

In [5]:
# Load 1st dataset: game2
game2 = spark.read.option('header', False).schema(game2_schema).csv('/user/tamng/jwht/SteamData/Games2_100milrows.csv')

In [6]:
# Check number of parition
game2.rdd.getNumPartitions()

39

In [7]:
# Count total number of observations
game2.count()

100000000

In [7]:
# Print Schema
game2.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: integer (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)



In [8]:
# Print out the first 2 lines of data
game2.head(2)

[Row(_c0='76561197960265729', _c1=10, _c2='"N,0,"2014-08-14 14:04:18"', _c3=None, _c4=None),
 Row(_c0='76561197960265729', _c1=20, _c2='"N,0,"2014-08-14 14:04:18"', _c3=None, _c4=None)]

---
__NOTES:__
>  - Problem: Any columns that contains Null Values was combined as one column (_c2). 
>  - Solution: Split to 2 dataframes, fomrat and merge together.
---

In [36]:
# Show data where c2 does not contain Null
game2.where(~ col("_c2").like('"N,%')).show(25)

+-----------------+------+----+-----+-------------------+
|              _c0|   _c1| _c2|  _c3|                _c4|
+-----------------+------+----+-----+-------------------+
|76561197960265730|    70|   1|  403|2014-08-14 14:04:45|
|76561197960265731|   570|  25|73072|2014-08-14 14:04:45|
|76561197960265731|  8930|1109| 1111|2014-08-14 14:04:45|
|76561197960265731|239220|   1|  133|2014-08-14 14:04:45|
|76561197960265733|    30|  42|   59|2014-08-14 14:04:45|
|76561197960265733|    70|  77| 1614|2014-08-14 14:04:45|
|76561197960265733| 71340|  18|   18|2014-08-14 14:04:45|
|76561197960265733|290810|  61|   61|2014-08-14 14:04:45|
|76561197960265738|   570|  25| 1108|2014-08-14 14:04:45|
|76561197960265738|247080| 953|  953|2014-08-14 14:04:45|
|76561197960265738|287980| 184|  184|2014-08-14 14:04:45|
|76561197960265742|   440| 328|21481|2014-08-14 14:04:41|
|76561197960265743|    10|   1|   41|2014-08-14 14:04:41|
|76561197960265744|203160| 275| 1449|2014-08-14 14:04:41|
|7656119796026

In [37]:
# Number of observation where c2 does not contain Null
game2.where(~ col("_c2").like('"N,%')).count()

1839672

In [38]:
# Number of observation where c2 contains Null
game2.where(col("_c2").like('"N,%')).count()

98160328

In [9]:
# game2_5cols
game2_5cols = game2.where(~ col("_c2").like('"N,%'))

In [10]:
# game2_3cols
game2_3cols = game2.where(col("_c2").like('"N,%'))

### 1.1 Format game2_3cols

In [11]:
game2_3cols.show(2)

+-----------------+---+--------------------+----+----+
|              _c0|_c1|                 _c2| _c3| _c4|
+-----------------+---+--------------------+----+----+
|76561197960265729| 10|"N,0,"2014-08-14 ...|null|null|
|76561197960265729| 20|"N,0,"2014-08-14 ...|null|null|
+-----------------+---+--------------------+----+----+
only showing top 2 rows



In [12]:
# Split column _c2 into 3 columns
split_col = F.split(game2_3cols['_c2'], ',')

# Merge split columns back to the orginal data and rename
game2_3cols = game2_3cols.withColumn('playtime_2weeks', split_col.getItem(0))
game2_3cols = game2_3cols.withColumn('playtime_forever', split_col.getItem(1))
game2_3cols = game2_3cols.withColumn('dateretrieved', split_col.getItem(2))

In [13]:
# Check column type
game2_3cols.dtypes

[('_c0', 'string'),
 ('_c1', 'int'),
 ('_c2', 'string'),
 ('_c3', 'string'),
 ('_c4', 'string'),
 ('playtime_2weeks', 'string'),
 ('playtime_forever', 'string'),
 ('dateretrieved', 'string')]

In [14]:
# Print out the first 3 lines
game2_3cols.head(3)

[Row(_c0='76561197960265729', _c1=10, _c2='"N,0,"2014-08-14 14:04:18"', _c3=None, _c4=None, playtime_2weeks='"N', playtime_forever='0', dateretrieved='"2014-08-14 14:04:18"'),
 Row(_c0='76561197960265729', _c1=20, _c2='"N,0,"2014-08-14 14:04:18"', _c3=None, _c4=None, playtime_2weeks='"N', playtime_forever='0', dateretrieved='"2014-08-14 14:04:18"'),
 Row(_c0='76561197960265729', _c1=30, _c2='"N,0,"2014-08-14 14:04:18"', _c3=None, _c4=None, playtime_2weeks='"N', playtime_forever='0', dateretrieved='"2014-08-14 14:04:18"')]

In [15]:
# Show 1st 5 lines
game2_3cols.show(5)

+-----------------+---+--------------------+----+----+---------------+----------------+--------------------+
|              _c0|_c1|                 _c2| _c3| _c4|playtime_2weeks|playtime_forever|       dateretrieved|
+-----------------+---+--------------------+----+----+---------------+----------------+--------------------+
|76561197960265729| 10|"N,0,"2014-08-14 ...|null|null|             "N|               0|"2014-08-14 14:04...|
|76561197960265729| 20|"N,0,"2014-08-14 ...|null|null|             "N|               0|"2014-08-14 14:04...|
|76561197960265729| 30|"N,0,"2014-08-14 ...|null|null|             "N|               0|"2014-08-14 14:04...|
|76561197960265729| 40|"N,0,"2014-08-14 ...|null|null|             "N|               0|"2014-08-14 14:04...|
|76561197960265729| 50|"N,0,"2014-08-14 ...|null|null|             "N|               0|"2014-08-14 14:04...|
+-----------------+---+--------------------+----+----+---------------+----------------+--------------------+
only showing top 5 

In [16]:
# Check distinct value of playtime_2weeks
game2_3cols.createOrReplaceTempView("game2_3cols")
spark.sql("SELECT DISTINCT(playtime_2weeks) FROM game2_3cols").show()

+---------------+
|playtime_2weeks|
+---------------+
|             "N|
+---------------+



__Adjust format of playtime_2weeks and dateretrieved feature__

In [17]:
# Remove " in playtime_2weeks and dateretrieved columns
game2_3cols = game2_3cols.withColumn('playtime_2weeks', regexp_replace('playtime_2weeks' ,'"',''))
game2_3cols = game2_3cols.withColumn('dateretrieved', regexp_replace('dateretrieved' ,'"',''))

In [19]:
# Change format to timestamp
game2_3cols = game2_3cols.withColumn('dateretrieved', (game2_3cols.dateretrieved).cast('timestamp'))

In [20]:
game2_3cols.head(2)

[Row(_c0='76561197960265729', _c1=10, _c2='"N,0,"2014-08-14 14:04:18"', _c3=None, _c4=None, playtime_2weeks='N', playtime_forever='0', dateretrieved=datetime.datetime(2014, 8, 14, 14, 4, 18)),
 Row(_c0='76561197960265729', _c1=20, _c2='"N,0,"2014-08-14 14:04:18"', _c3=None, _c4=None, playtime_2weeks='N', playtime_forever='0', dateretrieved=datetime.datetime(2014, 8, 14, 14, 4, 18))]

In [21]:
# Drop column _c2
game2_3cols = game2_3cols.drop("_c2","_c3","_c4")

# Rename _c0, _c1
game2_3cols = game2_3cols.withColumnRenamed('_c0', 'steam_id').withColumnRenamed('_c1', 'app_id')
game2_3cols.head(2)

[Row(steam_id='76561197960265729', app_id=10, playtime_2weeks='N', playtime_forever='0', dateretrieved=datetime.datetime(2014, 8, 14, 14, 4, 18)),
 Row(steam_id='76561197960265729', app_id=20, playtime_2weeks='N', playtime_forever='0', dateretrieved=datetime.datetime(2014, 8, 14, 14, 4, 18))]

In [22]:
# Change column type
game2_3cols = game2_3cols.withColumn("playtime_forever", game2_3cols["playtime_forever"].cast(IntegerType()))
game2_3cols = game2_3cols.withColumn('playtime_2weeks', regexp_replace('playtime_2weeks' ,'N', '0'))
game2_3cols = game2_3cols.withColumn("playtime_2weeks", game2_3cols["playtime_2weeks"].cast(IntegerType()))

In [23]:
game2_3cols.printSchema()

root
 |-- steam_id: string (nullable = true)
 |-- app_id: integer (nullable = true)
 |-- playtime_2weeks: integer (nullable = true)
 |-- playtime_forever: integer (nullable = true)
 |-- dateretrieved: timestamp (nullable = true)



### 1.2 Format game2_5cols

In [24]:
game2_5cols.show(2)

+-----------------+---+---+-----+-------------------+
|              _c0|_c1|_c2|  _c3|                _c4|
+-----------------+---+---+-----+-------------------+
|76561197960265730| 70|  1|  403|2014-08-14 14:04:45|
|76561197960265731|570| 25|73072|2014-08-14 14:04:45|
+-----------------+---+---+-----+-------------------+
only showing top 2 rows



In [25]:
game2_5cols.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: integer (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)



In [26]:
# Rename the columns
oldColumns = game2_5cols.schema.names
newColumns = ["steam_id", "app_id","playtime_2weeks","playtime_forever", "dateretrieved"]
  
game2_5cols = reduce(lambda game2_5cols, idx: game2_5cols.withColumnRenamed(oldColumns[idx], newColumns[idx]), range(len(oldColumns)), game2_5cols)
game2_5cols.printSchema()

root
 |-- steam_id: string (nullable = true)
 |-- app_id: integer (nullable = true)
 |-- playtime_2weeks: string (nullable = true)
 |-- playtime_forever: string (nullable = true)
 |-- dateretrieved: string (nullable = true)



In [27]:
# Change column type
game2_5cols = game2_5cols.withColumn("playtime_forever", game2_5cols["playtime_forever"].cast(IntegerType()))
game2_5cols = game2_5cols.withColumn("playtime_2weeks", game2_5cols["playtime_2weeks"].cast(IntegerType()))
game2_5cols = game2_5cols.withColumn('dateretrieved', (game2_5cols.dateretrieved).cast('timestamp'))

In [28]:
game2_5cols.printSchema()

root
 |-- steam_id: string (nullable = true)
 |-- app_id: integer (nullable = true)
 |-- playtime_2weeks: integer (nullable = true)
 |-- playtime_forever: integer (nullable = true)
 |-- dateretrieved: timestamp (nullable = true)



### 1.3 Merge game2_3cols & game2_5cols

In [29]:
game2_3cols.printSchema()

root
 |-- steam_id: string (nullable = true)
 |-- app_id: integer (nullable = true)
 |-- playtime_2weeks: integer (nullable = true)
 |-- playtime_forever: integer (nullable = true)
 |-- dateretrieved: timestamp (nullable = true)



In [30]:
game2_5cols.show(5)

+-----------------+------+---------------+----------------+-------------------+
|         steam_id|app_id|playtime_2weeks|playtime_forever|      dateretrieved|
+-----------------+------+---------------+----------------+-------------------+
|76561197960265730|    70|              1|             403|2014-08-14 14:04:45|
|76561197960265731|   570|             25|           73072|2014-08-14 14:04:45|
|76561197960265731|  8930|           1109|            1111|2014-08-14 14:04:45|
|76561197960265731|239220|              1|             133|2014-08-14 14:04:45|
|76561197960265733|    30|             42|              59|2014-08-14 14:04:45|
+-----------------+------+---------------+----------------+-------------------+
only showing top 5 rows



In [31]:
game2_3cols.show(5)

+-----------------+------+---------------+----------------+-------------------+
|         steam_id|app_id|playtime_2weeks|playtime_forever|      dateretrieved|
+-----------------+------+---------------+----------------+-------------------+
|76561197960265729|    10|              0|               0|2014-08-14 14:04:18|
|76561197960265729|    20|              0|               0|2014-08-14 14:04:18|
|76561197960265729|    30|              0|               0|2014-08-14 14:04:18|
|76561197960265729|    40|              0|               0|2014-08-14 14:04:18|
|76561197960265729|    50|              0|               0|2014-08-14 14:04:18|
+-----------------+------+---------------+----------------+-------------------+
only showing top 5 rows



In [80]:
game2_3cols.filter("playtime_forever != 0").show()

+-----------------+------+---------------+----------------+-------------------+
|         steam_id|app_id|playtime_2weeks|playtime_forever|      dateretrieved|
+-----------------+------+---------------+----------------+-------------------+
|76561197960265729|205790|              0|               1|2014-08-14 14:04:18|
|76561197960265730|    10|              0|             837|2014-08-14 14:04:45|
|76561197960265730|    20|              0|              27|2014-08-14 14:04:45|
|76561197960265730|    30|              0|               9|2014-08-14 14:04:45|
|76561197960265730|    40|              0|               5|2014-08-14 14:04:45|
|76561197960265730|    50|              0|               2|2014-08-14 14:04:45|
|76561197960265730|    80|              0|               6|2014-08-14 14:04:45|
|76561197960265730|   100|              0|              10|2014-08-14 14:04:45|
|76561197960265730|   220|              0|              14|2014-08-14 14:04:45|
|76561197960265730|   240|              

In [81]:
# No of players have playtime forever
game2_3cols.filter("playtime_forever != 0").count()

45250116

In [82]:
# No of players without playtime forever
game2_3cols.filter("playtime_forever == 0").count()

52910212

In [32]:
# Merge gam2_3cols and game2_5cols
game2_df = game2_3cols.union(game2_5cols)

In [84]:
game2_df.show(10)

+-----------------+------+---------------+----------------+-------------------+
|         steam_id|app_id|playtime_2weeks|playtime_forever|      dateretrieved|
+-----------------+------+---------------+----------------+-------------------+
|76561197960265729|    10|              0|               0|2014-08-14 14:04:18|
|76561197960265729|    20|              0|               0|2014-08-14 14:04:18|
|76561197960265729|    30|              0|               0|2014-08-14 14:04:18|
|76561197960265729|    40|              0|               0|2014-08-14 14:04:18|
|76561197960265729|    50|              0|               0|2014-08-14 14:04:18|
|76561197960265729|    60|              0|               0|2014-08-14 14:04:18|
|76561197960265729|    70|              0|               0|2014-08-14 14:04:18|
|76561197960265729|    80|              0|               0|2014-08-14 14:04:18|
|76561197960265729|   100|              0|               0|2014-08-14 14:04:18|
|76561197960265729|   130|              

In [85]:
# Check total observation to make sure we merge it correctly!
game2_df.count()

100000000

In [86]:
game2_df.filter("steam_id == 76561197960265729").show()

+-----------------+------+---------------+----------------+-------------------+
|         steam_id|app_id|playtime_2weeks|playtime_forever|      dateretrieved|
+-----------------+------+---------------+----------------+-------------------+
|76561197960265729|    10|              0|               0|2014-08-14 14:04:18|
|76561197960265729|    20|              0|               0|2014-08-14 14:04:18|
|76561197960265729|    30|              0|               0|2014-08-14 14:04:18|
|76561197960265729|    40|              0|               0|2014-08-14 14:04:18|
|76561197960265729|    50|              0|               0|2014-08-14 14:04:18|
|76561197960265729|    60|              0|               0|2014-08-14 14:04:18|
|76561197960265729|    70|              0|               0|2014-08-14 14:04:18|
|76561197960265729|    80|              0|               0|2014-08-14 14:04:18|
|76561197960265729|   100|              0|               0|2014-08-14 14:04:18|
|76561197960265729|   130|              

In [34]:
# Save game2_df
game2_df.write.csv('/user/tamng/jwht/CleanData/game2_df.csv', header = True)

In [91]:
!hdfs dfs -ls /user/tamng/jwht/SteamData

Java HotSpot(TM) 64-Bit Server VM warning: ignoring option MaxPermSize=512M; support was removed in 8.0
Found 7 items
-rwxrwxrwx   3 tamng tamng 5116243481 2020-05-09 22:16 /user/tamng/jwht/SteamData/Games2_100milrows.csv
-rwxrwxrwx   3 tamng tamng   79545914 2020-05-09 22:16 /user/tamng/jwht/SteamData/app_id_info_join.csv
-rwxrwxrwx   3 tamng tamng 1045968424 2020-05-09 22:17 /user/tamng/jwht/SteamData/friends_subset.csv
drwxr-xr-x   - tamng tamng          0 2020-05-16 21:28 /user/tamng/jwht/SteamData/game2_df.csv
-rwxrwxrwx   3 tamng tamng   83366366 2020-05-09 22:16 /user/tamng/jwht/SteamData/groups_subset.csv
-rwxrwxrwx   3 tamng tamng     568119 2020-05-09 22:16 /user/tamng/jwht/SteamData/metacritic_games.csv
-rwxrwxrwx   3 tamng tamng  899281516 2020-05-09 22:16 /user/tamng/jwht/SteamData/player_summaries_subset.csv


In [182]:
!hdfs dfs -ls /user/tamng/jwht/CleanData

Java HotSpot(TM) 64-Bit Server VM warning: ignoring option MaxPermSize=512M; support was removed in 8.0
Found 4 items
drwxrwxrwx   - tamng tamng          0 2020-05-16 23:12 /user/tamng/jwht/CleanData/friends.csv
drwxrwxrwx   - tamng tamng          0 2020-05-16 21:28 /user/tamng/jwht/CleanData/game2_df.csv
drwxrwxrwx   - tamng tamng          0 2020-05-16 22:58 /user/tamng/jwht/CleanData/game_app_info.csv
drwxrwxrwx   - tamng tamng          0 2020-05-16 23:15 /user/tamng/jwht/CleanData/groups.csv


In [191]:
game2_df.select([count(when(col('playtime_2weeks').isNull(), True))]).show()

+--------------------------------------------------------+
|count(CASE WHEN (playtime_2weeks IS NULL) THEN true END)|
+--------------------------------------------------------+
|                                                       0|
+--------------------------------------------------------+



___
### 2nd dataset: app_id_info.csv

__Problem: More columns than expected: 17 cols vs. 8 cols__

|No.|Column | Type | Description | Distinct_value| Notes |
|--|--|--|--|--|--|
|1|steamid|XX| The steam ID of the user in question|
|2|Title|XX|The Title of the app, as it appears to users|
|3|Type|XX| The type of the "app". Possible values include: "demo," "dlc," "game," "hardware," "mod," and "video." Game is the most common|
|4|Price|XX|The current price of the "app" on the Steam storefront, in US dollars. Free items have a price of 0.|
|5|Release_Date|XX|The date the "app" was made available via the Steam storefront. Note that apps released elsewhere originally and later published through steam carry the date of the Steam publish|
|6|Rating|XX|The rating of the "app" on Metacritic. Set to -1 if not applicable.|70
|7|Required_Age|XX|The MSRB or PEGI-assigned age requirement for viewing this game in the Steam storefront, and, by extension, clicking the button to purchase it.| 9 |
|8|Is_Multiplayer|XX|A value of either 0 or 1 indicating whether or not an "app" contains multiplayer content. Self-reported by developers.|2|
|9|achieveName|XX|The name of the achievement as it appears to players. As an internal value assigned by developers, its descriptiveness of the achievement varies.|130685| Double check name|
|10|achievePercent|XX|The percentage of players who have finished this achievement out of all total players who own this game.|X| |
|11|gamesDeveloper|XX|A developer of the app in question. Note that some apps have multiple developers and thus numerous distinct rows with the same appid are possible.|3624||
|12|gamesGenre|XX|A genre of the app in question. Note that most apps have multiple genres and thus numerous distinct rows with the same appid are possible.|20| |
|13|gamesPublisher|XX|A publisher of the app in question. Note that some apps have multiple publishers and thus numerous distinct rows with the same appid are possible.|2659| |




In [4]:
# Load app_id_info_join
app_id_info = spark.read.csv("/user/tamng/jwht/SteamData/app_id_info_join.csv", inferSchema=True, header=False)

In [5]:
app_id_info.rdd.getNumPartitions()

2

In [98]:
# Number of observations
app_id_info.count()

469811

In [99]:
app_id_info.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: double (nullable = true)
 |-- _c4: timestamp (nullable = true)
 |-- _c5: integer (nullable = true)
 |-- _c6: integer (nullable = true)
 |-- _c7: integer (nullable = true)
 |-- _c8: integer (nullable = true)
 |-- _c9: string (nullable = true)
 |-- _c10: double (nullable = true)
 |-- _c11: integer (nullable = true)
 |-- _c12: string (nullable = true)
 |-- _c13: integer (nullable = true)
 |-- _c14: string (nullable = true)
 |-- _c15: integer (nullable = true)
 |-- _c16: string (nullable = true)



In [6]:
app_id_info.show(3, truncate = True)

+---+--------------------+----+----+-------------------+---+---+---+---+----+---------+----+----------------+----+------+----+-----+
|_c0|                 _c1| _c2| _c3|                _c4|_c5|_c6|_c7|_c8| _c9|     _c10|_c11|            _c12|_c13|  _c14|_c15| _c16|
+---+--------------------+----+----+-------------------+---+---+---+---+----+---------+----+----------------+----+------+----+-----+
| 50|Half-Life: Opposi...|game|4.99|1999-11-01 00:00:00| -1|  0|  1| 50|ach0|0.0910381|  50|Gearbox Software|  50|Action|  50|Valve|
| 50|Half-Life: Opposi...|game|4.99|1999-11-01 00:00:00| -1|  0|  1| 50|ach1|0.0883079|  50|Gearbox Software|  50|Action|  50|Valve|
| 50|Half-Life: Opposi...|game|4.99|1999-11-01 00:00:00| -1|  0|  1| 50|ach2|0.0884962|  50|Gearbox Software|  50|Action|  50|Valve|
+---+--------------------+----+----+-------------------+---+---+---+---+----+---------+----+----------------+----+------+----+-----+
only showing top 3 rows



In [5]:
# Drop repeating cols
app_id_info = app_id_info.drop("_c8","_c11","_c13","_c15")

In [6]:
app_id_info.show(3, truncate = True)

+---+--------------------+----+----+-------------------+---+---+---+----+---------+----------------+------+-----+
|_c0|                 _c1| _c2| _c3|                _c4|_c5|_c6|_c7| _c9|     _c10|            _c12|  _c14| _c16|
+---+--------------------+----+----+-------------------+---+---+---+----+---------+----------------+------+-----+
| 50|Half-Life: Opposi...|game|4.99|1999-11-01 00:00:00| -1|  0|  1|ach0|0.0910381|Gearbox Software|Action|Valve|
| 50|Half-Life: Opposi...|game|4.99|1999-11-01 00:00:00| -1|  0|  1|ach1|0.0883079|Gearbox Software|Action|Valve|
| 50|Half-Life: Opposi...|game|4.99|1999-11-01 00:00:00| -1|  0|  1|ach2|0.0884962|Gearbox Software|Action|Valve|
+---+--------------------+----+----+-------------------+---+---+---+----+---------+----------------+------+-----+
only showing top 3 rows



In [7]:
# Rename the columns
oldColumns = app_id_info.schema.names
newColumns = ["app_id","title","type","price", "releasedDate", "rating", "requiredAge", "isMultiplayer", 
              "achieveName", "achievePercent", "gamesDeveloper", "gamesGenre", "gamesPublisher"]
  
app_id_info = reduce(lambda app_id_info, idx: app_id_info.withColumnRenamed(oldColumns[idx], newColumns[idx]), range(len(oldColumns)), app_id_info)
app_id_info.printSchema()

root
 |-- app_id: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- type: string (nullable = true)
 |-- price: double (nullable = true)
 |-- releasedDate: timestamp (nullable = true)
 |-- rating: integer (nullable = true)
 |-- requiredAge: integer (nullable = true)
 |-- isMultiplayer: integer (nullable = true)
 |-- achieveName: string (nullable = true)
 |-- achievePercent: double (nullable = true)
 |-- gamesDeveloper: string (nullable = true)
 |-- gamesGenre: string (nullable = true)
 |-- gamesPublisher: string (nullable = true)



In [8]:
# Save game2_df
app_id_info.write.csv('/user/tamng/jwht/CleanData/app_info.csv', header = True)

In [146]:
# Total rows after join the 2 dataframes: 7.6 bil rows
game_app_info.count()

7645637756

___
### 3. friends.csv

|No.|Column | Type | Description
|--|--|--|--|
|1|steam_id_a|str| The Steam ID of the user who's friend list was queried|
|2|steam_id_b|str| The Steam ID of the a user who is a friend of the user referenced by steamid_a|
|3|relationship|string| The type of relationship represented by this entry. Currently the only value used is "friend"|
|4|friend_since|timestamp| The date and time when the users in this entry became friends. Note that this field was added in 2009 and thus all frienships existing previous this date are recorded with the default unix timestamp (1970)|
|5|dateretrieved|timestamp| Timestamp when this friend list data was requested from the API|

****

_Step need to be done with this table:_
- Drop _c5
- Change column name
- Double check column type

In [9]:
# Load dataset
friends = spark.read.csv('/user/tamng/jwht/CleanData/friends.csv',\
                             inferSchema = True, header = True)

In [10]:
friends.printSchema()

root
 |-- _c0: long (nullable = true)
 |-- _c1: long (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: timestamp (nullable = true)
 |-- _c4: timestamp (nullable = true)
 |-- _c5: string (nullable = true)



In [155]:
friends.head(3)

[Row(_c0=76561197960265744, _c1=76561197960265731, _c2='friend', _c3=datetime.datetime(1969, 12, 31, 17, 0), _c4=datetime.datetime(2013, 5, 5, 1, 52, 30), _c5='N'),
 Row(_c0=76561197960265744, _c1=76561197960265733, _c2='friend', _c3=datetime.datetime(1969, 12, 31, 17, 0), _c4=datetime.datetime(2013, 5, 6, 15, 14, 15), _c5='N'),
 Row(_c0=76561197960265744, _c1=76561197960265738, _c2='friend', _c3=datetime.datetime(1969, 12, 31, 17, 0), _c4=datetime.datetime(2013, 5, 6, 15, 56, 24), _c5='N')]

In [11]:
# Drop col _c5
friends = friends.drop("_c5")

In [12]:
# Rename the header columns
newColumns = ["steam_id_a", "steam_id_b","relationship","friend_since", "dateretrieved"]
friends = rename_col(friends, newColumns)
friends.printSchema()

root
 |-- steam_id_a: long (nullable = true)
 |-- steam_id_b: long (nullable = true)
 |-- relationship: string (nullable = true)
 |-- friend_since: timestamp (nullable = true)
 |-- dateretrieved: timestamp (nullable = true)



In [13]:
friends_subset.show(3)

+-----------------+-----------------+------------+-------------------+-------------------+
|       steam_id_a|       steam_id_b|relationship|       friend_since|      dateretrieved|
+-----------------+-----------------+------------+-------------------+-------------------+
|76561197960265744|76561197960265731|      friend|1969-12-31 17:00:00|2013-05-05 01:52:30|
|76561197960265744|76561197960265733|      friend|1969-12-31 17:00:00|2013-05-06 15:14:15|
|76561197960265744|76561197960265738|      friend|1969-12-31 17:00:00|2013-05-06 15:56:24|
+-----------------+-----------------+------------+-------------------+-------------------+
only showing top 3 rows



In [14]:
# Save friends
friends_subset.write.csv('/user/tamng/jwht/CleanData/friends.csv', header= True)

In [192]:
!hdfs dfs -ls /user/tamng/jwht/CleanData

Java HotSpot(TM) 64-Bit Server VM warning: ignoring option MaxPermSize=512M; support was removed in 8.0
Found 4 items
drwxrwxrwx   - tamng tamng          0 2020-05-16 23:12 /user/tamng/jwht/CleanData/friends.csv
drwxrwxrwx   - tamng tamng          0 2020-05-16 21:28 /user/tamng/jwht/CleanData/game2_df.csv
drwxrwxrwx   - tamng tamng          0 2020-05-16 22:58 /user/tamng/jwht/CleanData/game_app_info.csv
drwxrwxrwx   - tamng tamng          0 2020-05-16 23:15 /user/tamng/jwht/CleanData/groups.csv


---
### 4. groups.csv

|No.|Column | Type | Description | Distinct_value|
|--|--|--|--|--|
|1|steam_id|str| The developer of the app in question|
|2|group_id|| A group ID for a group to which te user referenced by steamid belongs. Users may belong to more than one group|130748|
|3|dateretrieved|timestamp| Timestamp of the time when this game data was requested from the API|

****

_Step need to be done with this table:_
- Change column name
- Double check column type


In [15]:
groups = spark.read.csv('/user/tamng/jwht/SteamData/steamData_new/Groups.csv',\
                             inferSchema = True, header = False)

In [16]:
groups.printSchema()

root
 |-- _c0: long (nullable = true)
 |-- _c1: integer (nullable = true)
 |-- _c2: timestamp (nullable = true)



In [164]:
groups_subset.head(3)

[Row(_c0=76561197960265744, _c1=24, _c2=datetime.datetime(2013, 5, 6, 15, 56, 24)),
 Row(_c0=76561197960265744, _c1=206, _c2=datetime.datetime(2013, 5, 6, 15, 56, 24)),
 Row(_c0=76561197960265744, _c1=8484, _c2=datetime.datetime(2013, 5, 6, 15, 56, 24))]

In [17]:
# Rename the header columns
newColumns = ['steam_id', 'group_id', 'dateretrieved']
groups = rename_col(groups, newColumns)

root
 |-- steam_id: long (nullable = true)
 |-- group_id: integer (nullable = true)
 |-- dateretrieved: timestamp (nullable = true)



In [18]:
groups.show(3)

+-----------------+--------+-------------------+
|         steam_id|group_id|      dateretrieved|
+-----------------+--------+-------------------+
|76561197960265744|      24|2013-05-06 15:56:24|
|76561197960265744|     206|2013-05-06 15:56:24|
|76561197960265744|    8484|2013-05-06 15:56:24|
+-----------------+--------+-------------------+
only showing top 3 rows



In [19]:
# Save groups
groups.write.csv('/user/tamng/jwht/CleanData/groups.csv', header = True)

In [169]:
!hdfs dfs -ls /user/tamng/jwht/CleanData

Java HotSpot(TM) 64-Bit Server VM warning: ignoring option MaxPermSize=512M; support was removed in 8.0
Found 4 items
drwxrwxrwx   - tamng tamng          0 2020-05-16 23:12 /user/tamng/jwht/CleanData/friends.csv
drwxrwxrwx   - tamng tamng          0 2020-05-16 21:28 /user/tamng/jwht/CleanData/game2_df.csv
drwxrwxrwx   - tamng tamng          0 2020-05-16 22:58 /user/tamng/jwht/CleanData/game_app_info.csv
drwxrwxrwx   - tamng tamng          0 2020-05-16 23:15 /user/tamng/jwht/CleanData/groups.csv


___

### 5. player_summaries_subset.csv
    
|No.|Column | Type | Description
|--|--|--|--|
|1|steam_id|XX| The steam ID of the user in question|
|2|lastlogoff|XX| Timestamp of the time when this game data was requested from the API|
|3|primaryclan_id|XX| The groupid (Groups::groupid) of the group that the user has designated as their primary group|
|4|timecreated|XX| Timestamp of the time when the account was created|
|5|game_id|XX| If the user was in-game at the time of the API request, this value specifies which game they were running at the time|
|6|gameserverip|XX| If the user was in-game at the time of the request, and playing a game using Steam matchmaking, this value specifies the IP of the server they were connected to. Is otherwise set to "0.0.0.0:0"|
|7|loccountrycode|XX| ISO-3166 code for the country in which the user resides. Self-reported.|
|8|locstatecode|XX| State where the user resides. Self-reported.|
|9|loccity_id|XX| Internal Steam ID corresponding to the city where the user resides. Self-reported.|
|10|dateretrieved|XX| Timestamp of the time when this game data was requested from the API|


Work on this later

In [33]:
# Load 3rd dataset
player_summaries = spark.read.csv("/user/tamng/jwht/SteamData/player_summaries_subset.csv", inferSchema=True, header=False)

In [34]:
player_summaries.count()

1636741

In [35]:
player_summaries.dtypes

[('_c0', 'bigint'),
 ('_c1', 'string'),
 ('_c2', 'string'),
 ('_c3', 'string'),
 ('_c4', 'string'),
 ('_c5', 'string'),
 ('_c6', 'string'),
 ('_c7', 'string'),
 ('_c8', 'string'),
 ('_c9', 'string'),
 ('_c10', 'string'),
 ('_c11', 'string'),
 ('_c12', 'string'),
 ('_c13', 'string'),
 ('_c14', 'string'),
 ('_c15', 'string'),
 ('_c16', 'string'),
 ('_c17', 'string'),
 ('_c18', 'string'),
 ('_c19', 'string')]

In [36]:
player_summaries.head(2)

[Row(_c0=76561197960265744, _c1='ChrisB', _c2='http://steamcommunity.com/id/chrisb/', _c3='http://media.steampowered.com/steamcommunity/public/images/avatars/ff/ffa4cbf15ebf615e4a3e865455be1cbab714cfa8.jpg', _c4='http://media.steampowered.com/steamcommunity/public/images/avatars/ff/ffa4cbf15ebf615e4a3e865455be1cbab714cfa8_medium.jpg', _c5='http://media.steampowered.com/steamcommunity/public/images/avatars/ff/ffa4cbf15ebf615e4a3e865455be1cbab714cfa8_full.jpg', _c6='1', _c7='3', _c8='1', _c9='2013-02-13 20:00:35', _c10='1', _c11='Chris Bokitch', _c12='103582791429529892', _c13='2003-09-10 16:50:33', _c14='"N,"N', _c15='"N,"N', _c16='CA', _c17='BC', _c18='4832', _c19='2013-02-28 14:34:05'),
 Row(_c0=76561197960265744, _c1='ChrisB', _c2='http://steamcommunity.com/id/chrisb/', _c3='http://media.steampowered.com/steamcommunity/public/images/avatars/ff/ffa4cbf15ebf615e4a3e865455be1cbab714cfa8.jpg', _c4='http://media.steampowered.com/steamcommunity/public/images/avatars/ff/ffa4cbf15ebf615e4a3e

In [37]:
player_summaries.show(10)

+-----------------+------+--------------------+--------------------+--------------------+--------------------+---+---+---+-------------------+----+-------------+------------------+-------------------+-----+-----+----+----+----+-------------------+
|              _c0|   _c1|                 _c2|                 _c3|                 _c4|                 _c5|_c6|_c7|_c8|                _c9|_c10|         _c11|              _c12|               _c13| _c14| _c15|_c16|_c17|_c18|               _c19|
+-----------------+------+--------------------+--------------------+--------------------+--------------------+---+---+---+-------------------+----+-------------+------------------+-------------------+-----+-----+----+----+----+-------------------+
|76561197960265744|ChrisB|http://steamcommu...|http://media.stea...|http://media.stea...|http://media.stea...|  1|  3|  1|2013-02-13 20:00:35|   1|Chris Bokitch|103582791429529892|2003-09-10 16:50:33|"N,"N|"N,"N|  CA|  BC|4832|2013-02-28 14:34:05|
|7656119

In [38]:
# Count distinct number of steam_id in players
player_summaries.select('_c0').distinct().count()

371185

In [39]:
# Collect list of distinct steam_id from player data
steamid_player_list = player_summaries.select('_c0').distinct().collect()

# Turn the list to array
steamid_player_array = [int(row._c0) for row in steamid_player_list]

In [40]:
# Drop duplicates
player_summaries_dropdup = player_summaries.dropDuplicates() 
player_summaries_dropdup.show(3)

+-----------------+--------------+--------------------+--------------------+--------------------+--------------------+---+---+--------------------+-------------------+------------------+-------------------+-------------------+-----+--------------------+--------------------+--------------------+----+----+----+
|              _c0|           _c1|                 _c2|                 _c3|                 _c4|                 _c5|_c6|_c7|                 _c8|                _c9|              _c10|               _c11|               _c12| _c13|                _c14|                _c15|                _c16|_c17|_c18|_c19|
+-----------------+--------------+--------------------+--------------------+--------------------+--------------------+---+---+--------------------+-------------------+------------------+-------------------+-------------------+-----+--------------------+--------------------+--------------------+----+----+----+
|76561197971296720|      drew82ah|http://steamcommu...|http://media

In [172]:
#player_summaries_dropdup.head(100)

In [33]:
player_summaries_dropdup.count()

371185

In [34]:
# Count distinct 
player_summaries_dropdup.select('_c0').distinct().count()

371185

__Drop columns__

In [ ]:
player_summaries_dropdup = player_summaries_dropdup.drop('_c3','_c4','')

In [35]:
def filter_spark_dataframe_by_list(df, column_name, filter_list):
    """ Returns subset of df where df[column_name] is in filter_list """
    spark = SparkSession.builder.getOrCreate()
    filter_df = spark.createDataFrame(filter_list, df.schema[column_name].dataType)
    return df.join(filter_df, df[column_name] == filter_df["value"])

In [36]:
game_player_inc = filter_spark_dataframe_by_list(game2, 'steam_id', steamid_player_array)

In [37]:
game_player_inc.show(5)

+-----------------+------+---------------+----------------+-------------------+-----------------+
|         steam_id|app_id|playtime_2weeks|playtime_forever|      dateretrieved|            value|
+-----------------+------+---------------+----------------+-------------------+-----------------+
|76561197960268400|    10|              N|               0|2014-08-14 14:05:17|76561197960268400|
|76561197960268400|    20|              N|               0|2014-08-14 14:05:17|76561197960268400|
|76561197960268400|    30|              N|               0|2014-08-14 14:05:17|76561197960268400|
|76561197960268400|    40|              N|               0|2014-08-14 14:05:17|76561197960268400|
|76561197960268400|    50|              N|               0|2014-08-14 14:05:17|76561197960268400|
+-----------------+------+---------------+----------------+-------------------+-----------------+
only showing top 5 rows



In [44]:
player_summaries_dropdup.createOrReplaceTempView("player_summaries_dropdup")
spark.sql("SELECT DISTINCT(_c6) FROM player_summaries_dropdup").show()

+--------------------+
|                 _c6|
+--------------------+
|                   3|
|http://media.stea...|
|                   0|
|                   5|
|                   6|
|                  88|
|                   1|
|http://media.stea...|
|                   4|
|                   2|
|http://media.stea...|
|http://media.stea...|
+--------------------+



In [45]:
spark.sql("SELECT COUNT(DISTINCT(_c6)) FROM player_summaries_dropdup").show()

+-------------------+
|count(DISTINCT _c6)|
+-------------------+
|                 12|
+-------------------+



In [47]:
spark.sql("SELECT DISTINCT(_c7) FROM player_summaries_dropdup").show()

+--------------------+
|                 _c7|
+--------------------+
|"N,"2008-03-25 20...|
|"N,"2009-11-03 17...|
|                   3|
|"N,"2006-10-15 07...|
|"N,"2007-11-04 05...|
|"N,"2008-11-12 22...|
|                   0|
|"N,"2010-09-11 15...|
|"N,"2006-08-22 08...|
|"N,"2006-11-02 03...|
|"N,"2006-06-15 16...|
|                   1|
|"N,"2008-02-29 20...|
+--------------------+



In [46]:
spark.sql("SELECT COUNT(DISTINCT(_c7)) FROM player_summaries_dropdup").show()

+-------------------+
|count(DISTINCT _c7)|
+-------------------+
|                 13|
+-------------------+



In [55]:
spark.sql("SELECT DISTINCT(_c8) FROM player_summaries_dropdup").show()

+--------------------+
|                 _c8|
+--------------------+
|"N,"2011-04-26 13...|
|"N,"2013-03-04 05...|
|"N,"2006-09-18 22...|
|"N,"2012-11-09 06...|
|"N,"2012-10-03 12...|
|"N,"2013-02-13 12...|
|"N,"2008-03-02 15...|
|"N,"2007-10-12 21...|
|"N,"2010-10-05 10...|
|"N,"2006-10-22 15...|
|"N,"2010-06-27 15...|
|"N,"2008-10-08 13...|
|"N,"2011-09-05 19...|
|"N,"2013-01-03 12...|
|"N,"2010-01-26 17...|
|"N,"2007-06-22 10...|
|"N,"2008-01-19 15...|
|"N,"2007-10-13 19...|
|"N,"2010-10-20 04...|
|"N,"2008-01-06 11...|
+--------------------+
only showing top 20 rows



---
### APPENDIX

In [134]:
#app_id_info.select('gamesPublisher').distinct().count()
#app_id_info.select('gamesPublisher').distinct().show(10)
#app_id_info.select('achievePercent').show(10)
#app_id_info.select('achievePercent').sort('achievePercent', ascending = False).show()
#app_id_info.select('achievePercent').sort('achievePercent', ascending = True).show()

In [122]:
app_id_info.groupBy('achieveName').count().sort('count', ascending = False).show(50)

+--------------------+-----+
|         achieveName|count|
+--------------------+-----+
|                null| 1365|
| NEW_ACHIEVEMENT_1_0|  509|
| NEW_ACHIEVEMENT_1_1|  378|
| NEW_ACHIEVEMENT_1_2|  357|
| NEW_ACHIEVEMENT_1_3|  324|
| NEW_ACHIEVEMENT_1_4|  311|
| NEW_ACHIEVEMENT_1_6|  308|
| NEW_ACHIEVEMENT_1_7|  305|
| NEW_ACHIEVEMENT_1_5|  301|
| NEW_ACHIEVEMENT_1_8|  295|
| NEW_ACHIEVEMENT_1_9|  295|
|NEW_ACHIEVEMENT_1_10|  292|
|NEW_ACHIEVEMENT_1_11|  270|
|NEW_ACHIEVEMENT_1_12|  264|
|      ACHIEVEMENT_10|  255|
|NEW_ACHIEVEMENT_1_13|  246|
|      ACHIEVEMENT_11|  243|
|NEW_ACHIEVEMENT_1_15|  238|
|NEW_ACHIEVEMENT_1_14|  233|
|       ACHIEVEMENT_1|  229|
|      ACHIEVEMENT_12|  226|
|      ACHIEVEMENT_14|  215|
|      ACHIEVEMENT_13|  214|
|NEW_ACHIEVEMENT_1_16|  212|
|NEW_ACHIEVEMENT_1_17|  210|
|NEW_ACHIEVEMENT_1_19|  208|
|NEW_ACHIEVEMENT_1_18|  198|
|                  11|  193|
|      ACHIEVEMENT_15|  191|
|       ACHIEVEMENT_2|  188|
|      ACHIEVEMENT_16|  184|
|      ACHIEVE

In [22]:
# Number of distinct steamid (3.2mil)
game2_df.select('steam_id').distinct().count()

3263775

In [170]:
# Get a list of distinct steamid
#steamid_game_list = game2.select('steam_id').distinct().collect()
#steamid_game_list

In [171]:
# Turn the list to array
# steamid_game_array = [int(row.steam_id) for row in steamid_game_list]
# steamid_game_array